In [120]:
import numpy as np
import pandas as pd
from lightfm.data import Dataset
from scipy.io import mmwrite
from sklearn.model_selection import train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from sklearn.preprocessing import LabelEncoder


In [121]:
rating_df= pd.read_csv('../../data/train-test/implicit_data.csv')
outfit_df = pd.read_csv('../../data/train-test/outfit.csv')

In [122]:
# 상호작용이 적은 유저 필터링

# session_id를 기준으로 묶었을 때 각 그룹의 길이 구하기
grouped_counts = rating_df.groupby('session_id').size()

# 길이가 5 미만인 session_id를 삭제하기 위한 조건 생성
session_ids_to_remove = grouped_counts[grouped_counts < 5].index

# 조건에 해당하는 session_id를 삭제한 데이터프레임 생성
filtered_rating_df = rating_df[~rating_df['session_id'].isin(session_ids_to_remove)]

In [123]:
# # session_id를 라벨 인코딩
# label_encoder = LabelEncoder()
# filtered_rating_df['session_id'] = label_encoder.fit_transform(filtered_rating_df['session_id'])
filtered_rating_df.groupby('session_id').size()

session_id
02bfeca8-0440-42df-bba7-adbdea393ef4     6
033f88ff-2382-4742-a0a7-ca56b5c98571     7
049025e9-7327-4fcf-8b5b-3b129d119e8c    15
04a49069-ce63-4144-9eba-4c6a097281a0    43
052efac5-ad8c-4acd-bc41-1be5f1e83d1f    46
                                        ..
f58e3ebc-f51f-4318-b1e5-cb22379ce91f    13
f8dc0ea8-fc3a-435b-a93d-50c9f2d86396    11
fa9c15f8-26ea-4caf-a1a7-e7988e35c9a8     5
fb6abf78-226d-4a39-9fd3-9f3401091f06     6
fe3b84c2-5dba-4788-bf69-cb7b1ea170f0    49
Length: 124, dtype: int64

In [124]:
filtered_rating_df

,session_id,outfit_id,rating
0,7708c8e7-4292-4ff9-99b1-27be20427e42,83800,1
1,7708c8e7-4292-4ff9-99b1-27be20427e42,83791,1
2,7708c8e7-4292-4ff9-99b1-27be20427e42,84029,1
3,7708c8e7-4292-4ff9-99b1-27be20427e42,83706,1
4,7708c8e7-4292-4ff9-99b1-27be20427e42,83988,1
...,...,...,...
1693,ea15b529-afa8-4e44-aacc-f725a6837d4a,85516,1
1694,1c55ce42-2c94-44a3-a1b3-b037c87662ab,83999,1
1696,8995a6e7-a51c-463e-949e-add908c64d4b,86011,1
1697,da05e137-0516-4fa9-8297-a7335b504352,73342,1


In [125]:
# import matplotlib.pyplot as plt
# # 그래프 그리기
# plt.hist(grouped_counts, bins=len(grouped_counts), edgecolor='black')
# plt.xlabel('Size')
# plt.ylabel('Count')
# plt.title('Histogram of Session Size')
# plt.show()

In [126]:
# 유저별로 랜덤하게 1개의 데이터를 추출하여 train과 test 데이터프레임 생성
train_indices = []
test_indices = []
for _, group in filtered_rating_df.groupby('session_id'):
    if len(group) == 1:
        # 유저에게 데이터가 1개인 경우, train에 포함
        train_indices.append(group.index[0])
    else:
        # 랜덤하게 1개의 데이터를 test에 포함, 나머지는 train에 포함
        random_index = np.random.choice(group.index)
        test_indices.append(random_index)
        train_indices.extend(group.index.drop(random_index))

train_df = filtered_rating_df.loc[train_indices].reset_index(drop=True)
test_df = filtered_rating_df.loc[test_indices].reset_index(drop=True)

In [127]:
train_df['session_id'].nunique()

124

In [128]:
test_df['session_id'].nunique()

124

In [129]:
rating_df.groupby('session_id').size()

session_id
0178cc11-02c9-4c75-8089-9e8e1d9a31d5     1
017a246a-3a3a-43f2-862b-732b3e078d8f     1
02bfeca8-0440-42df-bba7-adbdea393ef4     6
033f88ff-2382-4742-a0a7-ca56b5c98571     7
049025e9-7327-4fcf-8b5b-3b129d119e8c    15
                                        ..
fa9c15f8-26ea-4caf-a1a7-e7988e35c9a8     5
fb6abf78-226d-4a39-9fd3-9f3401091f06     6
fb9c9692-1c50-4299-8d2c-5325d1577a90     1
fbddf83e-dc26-4064-9848-d1a211ac69f1     1
fe3b84c2-5dba-4788-bf69-cb7b1ea170f0    49
Length: 229, dtype: int64

In [130]:
# outfit 후보군 만들기
train_df

,session_id,outfit_id,rating
0,02bfeca8-0440-42df-bba7-adbdea393ef4,86535,1
1,02bfeca8-0440-42df-bba7-adbdea393ef4,87736,1
2,02bfeca8-0440-42df-bba7-adbdea393ef4,90119,1
3,02bfeca8-0440-42df-bba7-adbdea393ef4,83181,1
4,02bfeca8-0440-42df-bba7-adbdea393ef4,90093,1
...,...,...,...
1331,fe3b84c2-5dba-4788-bf69-cb7b1ea170f0,82414,1
1332,fe3b84c2-5dba-4788-bf69-cb7b1ea170f0,85081,1
1333,fe3b84c2-5dba-4788-bf69-cb7b1ea170f0,85523,1
1334,fe3b84c2-5dba-4788-bf69-cb7b1ea170f0,86709,1


In [131]:
# outfit_df = outfit_df[['outfit_id','reporter','gender','age','style','season', 'year']]

In [132]:
mean_age = int(outfit_df['age'].mean())
outfit_df['age'].fillna(mean_age)

0        23.0
1        19.0
2        22.0
3        25.0
4        24.0
         ... 
10424    21.0
10425    21.0
10426    19.0
10427    21.0
10428    21.0
Name: age, Length: 10429, dtype: float64

## Implicit -ALS MF

In [133]:
!pip install implicit


In [134]:
label_encoder = LabelEncoder()
train_df['session_id'] = label_encoder.fit_transform(train_df['session_id'])

In [112]:
train_df, test_df

(      session_id  outfit_id  rating
 0              0      86535       1
 1              0      80776       1
 2              0      90119       1
 3              0      83181       1
 4              0      90093       1
 ...          ...        ...     ...
 1331         123      82414       1
 1332         123      85081       1
 1333         123      85523       1
 1334         123      86709       1
 1335         123      83700       1
 
 [1336 rows x 3 columns],
                                session_id  outfit_id  rating
 0    02bfeca8-0440-42df-bba7-adbdea393ef4      87736       1
 1    033f88ff-2382-4742-a0a7-ca56b5c98571      83765       1
 2    049025e9-7327-4fcf-8b5b-3b129d119e8c      80424       1
 3    04a49069-ce63-4144-9eba-4c6a097281a0      78410       1
 4    052efac5-ad8c-4acd-bc41-1be5f1e83d1f      83488       1
 ..                                    ...        ...     ...
 119  f58e3ebc-f51f-4318-b1e5-cb22379ce91f      78826       1
 120  f8dc0ea8-fc3a-435b-a93d-50

In [135]:
train_df.nunique()

session_id     124
outfit_id     1190
rating           1
dtype: int64

In [136]:
1336/(124*1192)*100

0.9038752976834813

In [149]:
import pandas as pd
import numpy as np
import implicit
from scipy.sparse import coo_matrix

# train_df와 test_df 데이터프레임을 합친 상호작용 데이터 프레임 생성
interaction_data = pd.concat([train_df[['session_id', 'outfit_id']],
                              test_df[['session_id', 'outfit_id']]])

# 사용자와 아이템 ID 매핑
users = list(interaction_data['session_id'].unique())
items = list(interaction_data['outfit_id'].unique())
user_to_idx = {user: idx for idx, user in enumerate(users)}
item_to_idx = {item: idx for idx, item in enumerate(items)}

# train_df 데이터를 사용하여 user_item_matrix 생성 (1로 할당)
rows = [user_to_idx[user] for user in train_df['session_id']]
cols = [item_to_idx[item] for item in train_df['outfit_id']]
values = [1] * len(rows)

# 모든 사용자에 대해 1개의 행을 가지도록 user_item_matrix 생성
num_users = len(users)
num_items = len(items)
user_item_matrix = coo_matrix((values, (rows, cols)), shape=(num_users, num_items))
print(user_item_matrix)

user_item_matrix_csr = user_item_matrix.tocsr()

# ALS MF 모델 초기화
model = implicit.als.AlternatingLeastSquares(factors=50, iterations=10)

# 모델 학습 (희소 행렬 입력)
model.fit(user_item_matrix)

# 특정 사용자에 대한 아이템 추천 (예시로 사용자 1에 대해 추천)
user_id = 100  # 실제 사용자 ID로 변경해야 합니다.
user_idx = user_to_idx[user_id]

# 이미 상호작용한 아이템은 필터링하고, N개의 아이템을 추천합니다.
N = 5
recommended_items = model.recommend(user_idx, user_item_matrix_csr, N=N, filter_already_liked_items=True)

# 추천된 아이템 출력
print(f"사용자 ID: {user_id}")
for item_idx, score in recommended_items:
    item_id = items[item_idx]
    print(f'아이템 ID: {item_id}  Score: {score}')


TypeError: len() of unsized object

In [97]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD

# 데이터를 불러오기 위한 Reader 객체를 생성합니다.
reader = Reader(rating_scale=(0, 1))

# filtered_train_df 데이터프레임을 Surprise의 Dataset 객체로 변환합니다.
trainset = Dataset.load_from_df(filtered_train_df[['session_id', 'outfit_id', 'rating']].fillna(0), reader)

# DatasetAutoFolds를 Dataset으로 변환합니다.
trainset = trainset.build_full_trainset()

# SVD 모델을 생성하고 학습합니다.
model = SVD()
model.fit(trainset)

# 모든 아이디에 대해 상위 n개의 아이템을 추천합니다.
items_to_recommend = 50  # 추천할 아이템 개수

# 모든 사용자와 아이템의 id를 가져옵니다.
all_users = trainset.all_users()
all_items = set(trainset.all_items())  # Convert all_items to a set

# 새로운 사용자 ID (실제 사용자 ID로 대체해야 합니다.)
new_user_id = 101

# 새로운 사용자가 이미 평가한 아이템의 outfit_id를 가져옵니다.
try:
    # 사용자의 outfit_id를 가져옵니다.
    user_items_index = trainset.ur[trainset.to_inner_uid(new_user_id)]
    user_items = [filtered_train_df.at[trainset.to_raw_uid(new_user_id), 'outfit_id'] for (item_id, _) in user_items_index]
except ValueError:
    print(f"사용자 ID {new_user_id}는 기존 데이터에 존재하지 않습니다. 유효한 사용자 ID로 대체해주세요.")
    # 다른 유효한 사용자 ID로 대체해야 합니다.

# 모든 아이템 중에서 새로운 사용자가 평가하지 않은 아이템들을 구합니다.
items_to_predict = list(all_items - set(user_items))

# 예측한 평점을 기준으로 상위 n개의 아이템을 추천합니다.
predictions = [(iid, model.predict(new_user_id, iid).est) for iid in items_to_predict]

# 추천된 아이템 출력
print(f"새로운 사용자 ID: {new_user_id}")
for item_id, score in sorted(predictions, key=lambda x: x[1], reverse=True)[:items_to_recommend]:
    print(f'추천된 아이템 ID: {item_id}  평점: {score}')


새로운 사용자 ID: 101
추천된 아이템 ID: 0  평점: 1
추천된 아이템 ID: 1  평점: 1
추천된 아이템 ID: 2  평점: 1
추천된 아이템 ID: 3  평점: 1
추천된 아이템 ID: 4  평점: 1
추천된 아이템 ID: 5  평점: 1
추천된 아이템 ID: 6  평점: 1
추천된 아이템 ID: 7  평점: 1
추천된 아이템 ID: 8  평점: 1
추천된 아이템 ID: 9  평점: 1
추천된 아이템 ID: 10  평점: 1
추천된 아이템 ID: 11  평점: 1
추천된 아이템 ID: 12  평점: 1
추천된 아이템 ID: 13  평점: 1
추천된 아이템 ID: 14  평점: 1
추천된 아이템 ID: 15  평점: 1
추천된 아이템 ID: 16  평점: 1
추천된 아이템 ID: 17  평점: 1
추천된 아이템 ID: 18  평점: 1
추천된 아이템 ID: 19  평점: 1
추천된 아이템 ID: 20  평점: 1
추천된 아이템 ID: 21  평점: 1
추천된 아이템 ID: 22  평점: 1
추천된 아이템 ID: 23  평점: 1
추천된 아이템 ID: 24  평점: 1
추천된 아이템 ID: 25  평점: 1
추천된 아이템 ID: 26  평점: 1
추천된 아이템 ID: 27  평점: 1
추천된 아이템 ID: 28  평점: 1
추천된 아이템 ID: 29  평점: 1
추천된 아이템 ID: 30  평점: 1
추천된 아이템 ID: 31  평점: 1
추천된 아이템 ID: 32  평점: 1
추천된 아이템 ID: 33  평점: 1
추천된 아이템 ID: 34  평점: 1
추천된 아이템 ID: 35  평점: 1
추천된 아이템 ID: 36  평점: 1
추천된 아이템 ID: 37  평점: 1
추천된 아이템 ID: 38  평점: 1
추천된 아이템 ID: 39  평점: 1
추천된 아이템 ID: 40  평점: 1
추천된 아이템 ID: 41  평점: 1
추천된 아이템 ID: 42  평점: 1
추천된 아이템 ID: 43  평점: 1
추천된 아이템 ID: 44  평점: 1
추천된 

In [60]:
# !pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=1213284 sha256=a4f9f5b8b70f2a145800cbb734cd2634f2bb5c301625cb6f2a1aa97e3af38ddd
  Stored in directory: /opt/ml/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


# Light FM

In [38]:
outfit_df

,outfit_id,gender,age,reporter,tags,brands,region,occupation,style,date,season,year,tags_
0,64453,여성,NaN,화이트 컬러의 베이프 반팔 티셔츠와 와이드 데님 팬츠를 매칭한 룩입니다.,"반팔 티셔츠,가을,스트릿",unknown,부산/경남,전문직/프리랜서,스트릿,2020-09-17 00:00:00.000000,가을,2020,"반팔 티셔츠,,"
1,68758,여성,19.0,데님 스커트와 패딩 재킷을 매칭한 룩입니다.,"스니커즈,봄,캐주얼,나이키",나이키,홍대/신촌,학생,캐주얼,2021-02-24 00:00:00.000000,봄,2021,"스니커즈,,,"
2,71478,여성,22.0,블랙 컬러의 스커트와 화이트 셔츠를 매칭한 룩입니다.,"여성,셔츠,레더 스커트,워커,여름,캐주얼",unknown,홍대/신촌,사무직,캐주얼,2021-06-01 00:00:00.000000,여름,2021,",셔츠,레더 스커트,워커,,"
3,71480,여성,25.0,블랙 컬러의 슬리브리스 탑과 나이키 덩크 하이 스니커즈를 매칭한 룩입니다.,"여성,슬리브리스,트랙 팬츠,스니커즈,숄더 백,나이키,여름,로맨틱",나이키,홍대/신촌,학생,로맨틱,2021-06-01 00:00:00.000000,여름,2021,",슬리브리스,트랙 팬츠,스니커즈,숄더 백,,,"
4,71482,남성,24.0,블랙 컬러의 베스트와 나이키 덩크 '범고래' 를 매칭한\n 룩입니다.,"남성,비니,베스트,크로스 백,데님 팬츠,스니커즈,나이키,여름,스트릿",나이키,홍대/신촌,패션업,스트릿,2021-06-01 00:00:00.000000,여름,2021,",비니,베스트,크로스 백,데님 팬츠,스니커즈,,,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10424,92177,남성,21.0,"데님 반소매 셔츠와 블랙 데님 와이드 팬츠, 아디다스 스니커즈를 매치한 룩입니다.","여름,스트릿",unknown,홍대/신촌,학생,스트릿,2023-06-29 00:00:00.000000,여름,2023,","
10425,92178,여성,21.0,"화이트 컬러의 셔츠, 미니 스커트와 워커를 매치한 룩입니다.","여름,걸리시",unknown,홍대/신촌,학생,걸리시,2023-06-29 00:00:00.000000,여름,2023,","
10426,92179,여성,19.0,"그레이 컬러의 반소매 티셔츠, 화이트 컬러의 7부 팬츠를 매치한 룩입니다.","여름,스트릿",unknown,홍대/신촌,학생,스트릿,2023-06-29 00:00:00.000000,여름,2023,","
10427,92181,여성,21.0,"블랙 컬러의 시스루 카디건과 슬리브리스, 데님 팬츠를 매칭한 룩입니다.","여름,스트릿",unknown,홍대/신촌,학생,스트릿,2023-06-29 00:00:00.000000,여름,2023,","


In [39]:
import pandas as pd
from lightfm.data import Dataset
from scipy.io import mmwrite
from sklearn.model_selection import train_test_split
from lightfm import LightFM
from sklearn.preprocessing import LabelEncoder

# Assuming train_df and outfit_df are already defined

# session_id와 outfit_id를 라벨 인코딩하여 인덱스 형태로 변환
label_encoder = LabelEncoder()
train_df['session_id'] = label_encoder.fit_transform(train_df['session_id'])
ratings_source = [(train_df['session_id'][i], train_df['outfit_id'][i]) for i in range(train_df.shape[0])]

# Keep only the outfit_ids that are present in train_df
common_outfit_ids = train_df['outfit_id'].unique()
outfit_df = outfit_df[outfit_df['outfit_id'].isin(common_outfit_ids)]

outfit_df = outfit_df[['outfit_id', 'gender', 'age', 'style', 'season', 'year']]
outfit_df = outfit_df.fillna('unknown')  # NaN을 'unknown'으로 대체

# Create item features list using iterrows()
item_features_source = []
for index, row in outfit_df.iterrows():
    item_id = row['outfit_id']
    item_features = [row['gender'], row['age'], row['style'], row['season'], row['year']]
    item_features_source.append((item_id, item_features))



# Dataset 객체 생성
dataset = Dataset()

# 사용자와 아이템 라벨을 fit() 메서드를 통해 매핑
dataset.fit(users=train_df['session_id'].unique(),
            items=train_df['outfit_id'].unique(),
            item_features=outfit_df[outfit_df.columns[1:]].values.flatten()
            )

interactions, weights = dataset.build_interactions(ratings_source)
item_features = dataset.build_item_features(item_features_source)

# Save
mmwrite('../../data/train-test/interactions.mtx', interactions)
mmwrite('../../data/train-test/item_features.mtx', item_features)
mmwrite('../../data/train-test/weights.mtx', weights)


In [40]:
# Split Train, Test data
train_interactions, test_interactions, train_weights, test_weights = train_test_split(
    interactions, weights, test_size=0.1, random_state=42
)

# Convert to COO format
train_interactions = train_interactions.tocsr().tocoo()
test_interactions = test_interactions.tocsr().tocoo()
train_weights = train_weights.tocoo()
test_weights = test_weights.tocoo()

In [41]:
from hyperopt import fmin, hp, tpe, Trials

# Define Search Space
trials = Trials()
space = [hp.choice('no_components', range(10, 50, 10)),
         hp.uniform('learning_rate', 0.01, 0.05)]

In [42]:
# !pip install hyperopt

In [53]:
# Define Objective Function
def objective(params, train_interactions, test_interactions, train_weights, test_weights):
    no_components, learning_rate = params

    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(train_interactions,
              item_features=item_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False,
              user_features=None)

    test_precision = precision_at_k(model, test_interactions, k=5, item_features=item_features, user_features=None).mean()
    print("no_components: {}, learning_rate: {:.5f}, precision: {:.5f}".format(
        no_components, learning_rate, test_precision))
    
    # Return negative precision as we want to maximize it using the optimizer
    return -test_precision

# Call the objective function and pass the data
params = (10, 0.01)  # Replace with your desired parameter values
objective_result = objective(params, train_interactions, test_interactions, train_weights, test_weights)

ValueError: Incorrect number of features in user_features

In [50]:
# objective_result = objective((10, 0.01), train_interactions, test_interactions, train_weights, test_weights)

TypeError: objective() takes 1 positional argument but 5 were given

In [51]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

job exception: Incorrect number of features in user_features



  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]


ValueError: Incorrect number of features in user_features